In [5]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, execute
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.extensions.standard import HGate, SGate, SdgGate, XGate, YGate, IdGate, ZGate
import numpy as np
from scipy.stats import wasserstein_distance
from qiskit.quantum_info.states.measures import state_fidelity
import copy
import itertools
import cutter

def simulate_circ(circ, simulator='statevector_simulator'):
    backend = BasicAer.get_backend(simulator)
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputprob = [np.power(abs(x),2) for x in outputstate]
    return outputprob

q = QuantumRegister(3,'q')
original_circ = QuantumCircuit(q)
original_circ.h(0)
original_circ.h(1)
original_circ.cx(0,1)
original_circ.cx(1,2)
original_circ.t(1)
original_circ.y(1)
original_circ.h(0)
original_circ.y(2)
original_circ_prob = simulate_circ(original_circ)
print('original circuit:')
print(original_circ)
print('uncut measurement:', original_circ_prob)
print('*'*100)

positions = [(q[1],1)]
clusters, complete_path_map, K, d = cutter.cut_circuit(original_circ, positions)

print('Complete path map:')
[print(x, ':',complete_path_map[x]) for x in complete_path_map]
for idx, x in enumerate(clusters):
    print('cluster %d circuit:'%idx)
    print(x)
print('*'*50)

original circuit:
        ┌───┐     ┌───┐          
q_0: |0>┤ H ├──■──┤ H ├──────────
        ├───┤┌─┴─┐└───┘┌───┐┌───┐
q_1: |0>┤ H ├┤ X ├──■──┤ T ├┤ Y ├
        └───┘└───┘┌─┴─┐├───┤└───┘
q_2: |0>──────────┤ X ├┤ Y ├─────
                  └───┘└───┘     
uncut measurement: [0.4999999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4999999999999999, 0.0]
****************************************************************************************************
Complete path map:
Qubit(QuantumRegister(3, 'q'), 0) : [(0, Qubit(QuantumRegister(2, 'q'), 0))]
Qubit(QuantumRegister(3, 'q'), 1) : [(0, Qubit(QuantumRegister(2, 'q'), 1)), (1, Qubit(QuantumRegister(1, 'cutQ'), 0))]
Qubit(QuantumRegister(3, 'q'), 2) : [(1, Qubit(QuantumRegister(1, 'q'), 0))]
cluster 0 circuit:
        ┌───┐     ┌───┐
q_0: |0>┤ H ├──■──┤ H ├
        ├───┤┌─┴─┐└───┘
q_1: |0>┤ H ├┤ X ├─────
        └───┘└───┘     
cluster 1 circuit:
           ┌───┐┌───┐     
   q_0: |0>┤ X ├┤ Y ├─────
           └─┬─┘├───┤┌───┐
cutQ_0: |0>──■──┤ T ├

In [6]:
all_s = list(itertools.product(range(1,9),repeat=len(positions)))
reconstructed = [0 for x in range(len(original_circ_prob))]
for s in all_s:
    cluster_0_copy = copy.deepcopy(clusters[0])
    cluster_0_dag = circuit_to_dag(cluster_0_copy)
    O_qubit = cluster_0_copy.qubits[1]
    
    cluster_1_copy = copy.deepcopy(clusters[1])
    cluster_1_dag = circuit_to_dag(cluster_1_copy)
    rho_qubit = cluster_1_copy.qubits[1]
    
    if s[0]==2 or s[0]==8:
        cluster_1_dag.apply_operation_front(op=XGate(),qargs=[rho_qubit],cargs=[])
    elif s[0]==3:
        cluster_0_dag.apply_operation_back(op=HGate(),qargs=[O_qubit])
        cluster_1_dag.apply_operation_front(op=HGate(),qargs=[rho_qubit],cargs=[])
    elif s[0]==4:
        cluster_0_dag.apply_operation_back(op=HGate(),qargs=[O_qubit])
        cluster_1_dag.apply_operation_front(op=HGate(),qargs=[rho_qubit],cargs=[])
        cluster_1_dag.apply_operation_front(op=XGate(),qargs=[rho_qubit],cargs=[])
    elif s[0]==5:
        cluster_0_dag.apply_operation_back(op=SdgGate(),qargs=[O_qubit])
        cluster_0_dag.apply_operation_back(op=HGate(),qargs=[O_qubit])
        cluster_1_dag.apply_operation_front(op=SGate(),qargs=[rho_qubit],cargs=[])
        cluster_1_dag.apply_operation_front(op=HGate(),qargs=[rho_qubit],cargs=[])
    elif s[0]==6:
        cluster_0_dag.apply_operation_back(op=SdgGate(),qargs=[O_qubit])
        cluster_0_dag.apply_operation_back(op=HGate(),qargs=[O_qubit])
        cluster_1_dag.apply_operation_front(op=SGate(),qargs=[rho_qubit],cargs=[])
        cluster_1_dag.apply_operation_front(op=HGate(),qargs=[rho_qubit],cargs=[])
        cluster_1_dag.apply_operation_front(op=XGate(),qargs=[rho_qubit],cargs=[])
    
    cluster_0_circ = dag_to_circuit(cluster_0_dag)
    cluster_0_meas = simulate_circ(cluster_0_circ)
    
    cluster_1_circ = dag_to_circuit(cluster_1_dag)
    cluster_1_meas = simulate_circ(cluster_1_circ)
    
    t_s = np.kron(cluster_0_meas,cluster_1_meas)
    if s[0]!=1 and s[0]!=2:
        t_s = [t_s[i]-t_s[i+8] for i in range(8)]
    else:
        t_s = [t_s[i]+t_s[i+8] for i in range(8)]
    
    if s[0]==4 or s[0]==6 or s[0]==8:
        t_s = [-1/2*x for x in t_s]
    else:
        t_s = [1/2*x for x in t_s]
    
    t_s_ordered = [0 for x in t_s]
    t_s_ordered[0] = t_s[0]
    t_s_ordered[1] = t_s[4]
    t_s_ordered[2] = t_s[2]
    t_s_ordered[3] = t_s[6]
    t_s_ordered[4] = t_s[1]
    t_s_ordered[5] = t_s[5]
    t_s_ordered[6] = t_s[3]
    t_s_ordered[7] = t_s[7]
    for i in range(len(reconstructed)):
        reconstructed[i] += t_s_ordered[i]
    
print(reconstructed)
print(original_circ_prob)
wasserstein_distance(original_circ_prob,reconstructed)

[0.4999999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0]
[0.4999999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4999999999999999, 0.0]


1.3877787807814457e-17